# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords',])
from nltk.corpus import stopwords
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sqlalchemy import create_engine
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, accuracy_score, precision_score, recall_score
import unicodedata
from nltk.tokenize import WhitespaceTokenizer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('disater_response', engine)
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def load_data():
    df = pd.read_sql_table('disater_response', engine)
    X = df['message']
    y = df.iloc[:,4:]
    
    
    
    return X, y

def tokenize(text):
    # Tokenize text using word_tokenize
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # Remove stopwords and perform additional normalization
    processed_tokens = []
    for token in tokens:
        # Convert token to lowercase and remove punctuation
        token = token.lower().strip('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
        # Remove accents
        token = unicodedata.normalize('NFKD', token).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        # Lemmatize token
        token = lemmatizer.lemmatize(token)
        # Remove stopwords and append token to processed tokens if not empty
        if token not in stopwords.words('english') and token != '':
            processed_tokens.append(token)
    return processed_tokens


 


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
from sklearn.multioutput import MultiOutputClassifier
# Build machine learning pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Split data into train and test sets
X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train pipeline
pipeline.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
 #Report the f1 score, precision, and recall for each output category

#for i, column in enumerate(y.columns):
   # print("Category:", column)
   # print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

# Define the categories based on your dataset columns

categories = y.columns.tolist()

y_pred = pipeline.predict(X_test)

print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=categories))

                        precision    recall  f1-score   support

               related       0.78      0.45      0.57       856
               request       0.00      0.00      0.00        32
                 offer       0.73      0.59      0.65      2156
           aid_related       0.64      0.12      0.20       394
          medical_help       0.59      0.07      0.13       260
      medical_products       0.67      0.07      0.12       153
     search_and_rescue       0.33      0.01      0.03        74
              security       0.78      0.08      0.15       170
              military       0.00      0.00      0.00         0
           child_alone       0.82      0.38      0.52       315
                 water       0.83      0.46      0.59       579
                  food       0.77      0.28      0.41       465
               shelter       0.77      0.12      0.20        85
              clothing       1.00      0.02      0.03       121
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:

# Define pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

# Define parameters for grid search
parameters = {
    'tfidf__smooth_idf': [True, False],
    'clf__estimator__n_estimators': [50, 100],
}

# Initialize GridSearchCV
cv = GridSearchCV(pipeline, parameters, cv=2, verbose=2)

# Fit the GridSearchCV object
cv.fit(X_train, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] clf__estimator__n_estimators=50, tfidf__smooth_idf=True .........
[CV]  clf__estimator__n_estimators=50, tfidf__smooth_idf=True, total= 3.5min
[CV] clf__estimator__n_estimators=50, tfidf__smooth_idf=True .........


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.4min remaining:    0.0s


[CV]  clf__estimator__n_estimators=50, tfidf__smooth_idf=True, total= 3.6min
[CV] clf__estimator__n_estimators=50, tfidf__smooth_idf=False ........
[CV]  clf__estimator__n_estimators=50, tfidf__smooth_idf=False, total= 3.5min
[CV] clf__estimator__n_estimators=50, tfidf__smooth_idf=False ........
[CV]  clf__estimator__n_estimators=50, tfidf__smooth_idf=False, total= 3.4min
[CV] clf__estimator__n_estimators=100, tfidf__smooth_idf=True ........
[CV]  clf__estimator__n_estimators=100, tfidf__smooth_idf=True, total= 5.2min
[CV] clf__estimator__n_estimators=100, tfidf__smooth_idf=True ........
[CV]  clf__estimator__n_estimators=100, tfidf__smooth_idf=True, total= 5.1min
[CV] clf__estimator__n_estimators=100, tfidf__smooth_idf=False .......
[CV]  clf__estimator__n_estimators=100, tfidf__smooth_idf=False, total= 5.2min
[CV] clf__estimator__n_estimators=100, tfidf__smooth_idf=False .......
[CV]  clf__estimator__n_estimators=100, tfidf__smooth_idf=False, total= 5.3min


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 42.8min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__smooth_idf': [True, False], 'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
y_pred=cv.predict(X_test)
print(accuracy_score(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred])))

0.412090007628


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
cv.best_params_

{'clf__estimator__n_estimators': 100, 'tfidf__smooth_idf': True}

### 9. Export your model as a pickle file

In [ ]:
import pickle

# Define the filename for the pickle file
filename = 'model.pkl'

# Export the trained model as a pickle file
with open('model.pkl', 'wb') as file:
    pickle.dump(cv, file)


### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.